# Project Part 3

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/entrylevelcs/CS39AA-Project/blob/main/project_part3.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/entrylevelcs/CS39AA-Project/blob/main/project_part3.ipynb)


## 1. Introduction/Background

For this part of the project we are using the bert based pretrained model and training it using our two data sets to see how accurate we can get the predictions to be.

## 2. Using pretrained models to improve accuracy

In [1]:
# import all of the python modules/packages you'll need here
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,  TrainingArguments, Trainer
from datasets import Dataset, load_metric
import os
os.environ["WANDB_DISABLED"] = "true"
# ...

c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Get the data set from real steam reviews. The specific set of data that I am using for this notebook comes from https://www.kaggle.com/datasets/andrewmvd/steam-reviews/ but is just a sample of 25000 from the entire set.

In [2]:
human_data = 'https://raw.githubusercontent.com/entrylevelcs/CS39AA-Project/main/human_dataset.csv'
df = pd.read_csv(human_data)
df = df[df["review_text"].notnull()]
sample_size = len(df)

Get the data set that was generated by chatgpt. This review data started as only being about CS:GO but has been expanded to be more general and talk about other games.

In [3]:
ai_data = 'https://raw.githubusercontent.com/entrylevelcs/CS39AA-Project/main/gpt3.5_generated_data.csv'
df2 = pd.read_csv(ai_data)

We need to reformat the data into forms that are usable to the model. One thing we need to do is reclassify our labels since the model marks "-1" as a wrong label. In order to do this we replace the recommended labels, 1, with 0 and the not recommende labels, -1 with 1.

In [4]:
df = df.rename(columns={"review_text": "text", "review_score": "label"})
df2 = df2.rename(columns={"Review": "text", " Sentiment": "label"})
classes = df.label.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
df['label'] = df['label'].apply(lambda x: class_tok2idx[x])
classes = df2.label.unique().tolist()
class_tok2idx = dict((v, k) for k, v in enumerate(classes))
df2['label'] = df2['label'].apply(lambda x: class_tok2idx[x])

We get some numbers related to the two data sets sizes so that the data going into both training models are the same.

In [5]:
ai_data_size = df2.size
sample_size = df.size
proportion = 1 - ((sample_size-ai_data_size)/sample_size)

Initialize our tokenizer and models.

In [6]:
real_data_raw = Dataset.from_pandas(df[['label', 'text']])
simulated_data_raw = Dataset.from_pandas(df2)
MODEL_NAME = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens({'pad_token': '<pad>'})
realModel = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, max_length=55, output_attentions=False, output_hidden_states=False)
simulatedModel = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, max_length=55, output_attentions=False, output_hidden_states=False)
realModel.resize_token_embeddings(len(tokenizer))
simulatedModel.resize_token_embeddings(len(tokenizer))

c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
c:\Users\Muajeh Lee\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN th

Embedding(28997, 768)

Tokenize our data sets so that they can be used in the models.

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=55)

real_data = real_data_raw.map(tokenize_function, batched=True)
simulated_data = simulated_data_raw.map(tokenize_function, batched=True)

Map:   0%|          | 0/24971 [00:00<?, ? examples/s]

Map:   0%|          | 0/2496 [00:00<?, ? examples/s]

Split the real data into the training data and the evaluation data set. Both the simulated and real training data sets will use the real_data_eval data set as the evaluation data sets. With this split both our simulated training data set and our real training data set are the same size.

In [8]:
train_prop = proportion
real_data_train = real_data.select(range(int(len(real_data)*train_prop)))
real_data_eval = real_data.select(range(int(len(real_data)*train_prop), len(real_data)))

Set up our real model that is trained on the real data.

In [9]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

real_training_args = TrainingArguments(num_train_epochs=5,
                                  do_train=True,
                                  report_to=None,
                                  output_dir="real",
                                  evaluation_strategy="steps",
                                  eval_steps=50,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

real_trainer = Trainer(model = realModel, 
                  args = real_training_args,
                  train_dataset = real_data_train, 
                  eval_dataset = real_data_eval,
                  compute_metrics = compute_metrics,
)

C:\Users\Muajeh Lee\AppData\Local\Temp\ipykernel_4880\3854264034.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Set up our simulated model that is trained on simulated data.

In [10]:
sim_training_args = TrainingArguments(num_train_epochs=5,
                                  do_train=True,
                                  report_to=None,
                                  output_dir="simulated",
                                  evaluation_strategy="steps",
                                  eval_steps=50,
                                  learning_rate=1e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32)

sim_trainer = Trainer(model = simulatedModel, 
                  args = sim_training_args,
                  train_dataset = simulated_data, 
                  eval_dataset = real_data_eval,
                  compute_metrics = compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
if torch.cuda.is_available():
    device = "cuda:0"
    print("Using GPU")
else: 
    device = "cpu"


Using GPU


Train our real data model.

In [12]:
realModel.to(device)
torch.set_grad_enabled(True)
real_trainer.train()
real_trainer.evaluate()

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/755 [00:00<?, ?it/s]

{'eval_loss': 0.45227646827697754, 'eval_accuracy': 0.8186412593206297, 'eval_runtime': 51.8747, 'eval_samples_per_second': 465.352, 'eval_steps_per_second': 14.554, 'epoch': 1.92}


  0%|          | 0/755 [00:00<?, ?it/s]

{'eval_loss': 0.40105804800987244, 'eval_accuracy': 0.8186412593206297, 'eval_runtime': 50.3002, 'eval_samples_per_second': 479.918, 'eval_steps_per_second': 15.01, 'epoch': 3.85}
{'train_runtime': 130.5541, 'train_samples_per_second': 31.826, 'train_steps_per_second': 0.996, 'train_loss': 0.39706529470590446, 'epoch': 5.0}


  0%|          | 0/755 [00:00<?, ?it/s]

{'eval_loss': 0.3831833004951477,
 'eval_accuracy': 0.821913835956918,
 'eval_runtime': 49.712,
 'eval_samples_per_second': 485.597,
 'eval_steps_per_second': 15.187,
 'epoch': 5.0}

Train our simulated data model.

In [13]:
simulatedModel.to(device)
sim_trainer.train()
sim_trainer.evaluate()

  0%|          | 0/390 [00:00<?, ?it/s]

  0%|          | 0/755 [00:00<?, ?it/s]

{'eval_loss': 0.48819759488105774, 'eval_accuracy': 0.7671085335542668, 'eval_runtime': 50.324, 'eval_samples_per_second': 479.692, 'eval_steps_per_second': 15.003, 'epoch': 0.64}


  0%|          | 0/755 [00:00<?, ?it/s]

{'eval_loss': 0.6519426107406616, 'eval_accuracy': 0.7659486329743165, 'eval_runtime': 47.5334, 'eval_samples_per_second': 507.854, 'eval_steps_per_second': 15.884, 'epoch': 1.28}


...

{'eval_loss': 1.190108299255371, 'eval_accuracy': 0.6944904722452361, 'eval_runtime': 51.7424, 'eval_samples_per_second': 466.542, 'eval_steps_per_second': 14.592, 'epoch': 3.85}


  0%|          | 0/755 [00:00<?, ?it/s]

{'eval_loss': 1.1267844438552856, 'eval_accuracy': 0.7207125103562552, 'eval_runtime': 49.8369, 'eval_samples_per_second': 484.38, 'eval_steps_per_second': 15.149, 'epoch': 4.49}
{'train_runtime': 428.9347, 'train_samples_per_second': 29.095, 'train_steps_per_second': 0.909, 'train_loss': 0.10714641473232171, 'epoch': 5.0}


  0%|          | 0/755 [00:00<?, ?it/s]

{'eval_loss': 1.214277982711792,
 'eval_accuracy': 0.7009942004971003,
 'eval_runtime': 49.9315,
 'eval_samples_per_second': 483.462,
 'eval_steps_per_second': 15.121,
 'epoch': 5.0}